# Installing & Importing Required Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install kagglehub
!pip install pyLSHash
!pip install faiss-cpu


import kagglehub
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import shutil
import os
import pandas as pd
import numpy as np
from pyLSHash import LSHash
import faiss
import re
import torch
from time import time
from pyLSHash import SimHash, hamming


  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy-hash: filename=fuzzy_hash-0.0.3-cp310-cp310-linux_x86_64.whl size=22899 sha256=acb7c896fce6fe2e76d0e419f64e3d311e240602d9f5fbcf8bc04a1bab7efd14
  Stored in directory: /root/.cache/pip/wheels/84/86/fd/a0b01e8f494ee10c4d32d05bca30cd0aaf2090aaaa3ac8cdd1
Successfully built fuzzy-hash
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 24.2 MB/s eta 0:00:00


# Loading Model

## PUBMED BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")


config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

## Clinical BERT

In [2]:

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

# Testing Tokenizer & Output Embeddings

## Case 1

In [3]:
inputs = tokenizer("Fungal infection is a common skin condition caused by fungi.", return_tensors="pt")
outputs = model(**inputs, output_hidden_states=True)


In [4]:
hidden_states = outputs.hidden_states
last_layer_embeddings = hidden_states[-1]

In [5]:
inputs

{'input_ids': tensor([[   101,  41807,  17026,  55223,  10124,    169,  14624,  40564,  24713,
          19513,  10155, 107950,    119,    102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
last_layer_embeddings

tensor([[[-0.0075, -0.1032,  0.4786,  ..., -0.0054,  0.0832, -0.2542],
         [-0.3233, -0.5111,  0.6904,  ..., -0.8858, -0.0309, -0.2969],
         [-0.0361, -0.3336,  1.2755,  ...,  0.2745, -0.0976, -0.4397],
         ...,
         [-0.5215, -0.1644,  0.7890,  ...,  0.0165,  0.1556, -0.3001],
         [-0.3448, -0.1667,  1.0636,  ..., -0.0472,  0.2363, -0.3965],
         [ 0.1714, -0.2603,  0.6884,  ..., -0.2345, -0.0743, -0.1550]]],
       grad_fn=<NativeLayerNormBackward0>)

In [7]:
tokenizer.decode(inputs["input_ids"][0])

'[CLS] fungal infection is a common skin condition caused by fungi. [SEP]'

## Case 2

In [ ]:
inputs = tokenizer("Fungal infection is a common skin disease caused by fungi.", return_tensors="pt")
outputs = model(**inputs, output_hidden_states=True)
hidden_states = outputs.hidden_states
last_layer_embeddings = hidden_states[-1]
inputs

{'input_ids': tensor([[   101,  41807,  17026,  55223,  10124,    169,  14624,  40564,  21911,
          19513,  10155, 107950,    119,    102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
cls_embedding = last_layer_embeddings[:, 0, :]
cls_embedding.shape, last_layer_embeddings.shape

(torch.Size([1, 768]), torch.Size([1, 14, 768]))

# Generating Embeddings

### Downloading Dataset From Kaggle

In [29]:
path = kagglehub.dataset_download("samoyedzzzzz/34000-medicine-details")
# noorsaeed/medicine-recommendation-system-dataset (Dataset1 path)

100%|██████████| 59.8k/59.8k [00:00<00:00, 812kB/s]

Extracting files...


In [30]:
source_path = path

destination_path = "/content/Dataset2"
os.makedirs(destination_path, exist_ok=True)

for file_name in os.listdir(source_path):
    full_file_name = os.path.join(source_path, file_name)
    if os.path.isfile(full_file_name):
        shutil.move(full_file_name, destination_path)

# Embedding Generation

## Generate Function

In [10]:
def meanpooling(output, mask):
    """
    Mean Pooling - Take attention mask into account for correct averaging.
    """
    embeddings = output[0]
    mask = mask.to(embeddings.device)
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


In [11]:
def generate_embeddings(
    texts, tokenizer, model, query=False,
    max_length=128, batch_size=32, Bert=None):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    all_embeddings = []
    if query:
        prefix = "Medication for "
        texts = [f"{prefix}{text}" for text in texts]

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        )

        model_inputs = {
            "input_ids": inputs["input_ids"].to(device),
            "attention_mask": inputs["attention_mask"].to(device),
            "output_hidden_states": True,
            "return_dict": True,
        }

        if "token_type_ids" in inputs:
            model_inputs["token_type_ids"] = inputs["token_type_ids"].to(device)

        with torch.no_grad():
            outputs = model(**model_inputs)
        if Bert == 'pubmed':
            embeddings = meanpooling(outputs, inputs["attention_mask"]).cpu().numpy()
        else:
            embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()
        all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)


## Data Preparation

### Dataset 2 loading

In [12]:
df = pd.read_csv("/content/Dataset2/dataset.csv")
important_columns = ['Name', 'Uses']
df = df[important_columns].fillna("")

def split_side_effects(text):
    return re.findall(r'[A-Z][a-z]*|[a-z]+|\(.*?\)|\S+', text)

df['Uses'] = df['Uses'].apply(lambda text: ' '.join(split_side_effects(text)))
df['input_text'] = "Drug Name is " + df['Name'] + " Which is used for " + df['Uses']
df['input_text'].head()


,input_text
0,Drug Name is Aceloflam Plus Tablet Which is us...
1,Drug Name is Aceloflam Plus Tablet Which is us...
2,Drug Name is Aceloflam Plus Tablet Which is us...
3,Drug Name is Aceloflam Plus Tablet Which is us...
4,Drug Name is Adriamycin 50mg Injection Which i...


In [13]:
df = df.groupby('Uses').filter(lambda group: len(group) > 1)
df.shape

(34109, 3)

In [14]:
df = df.sample(frac=1, random_state=42)
new_df = df.sample(n=200, random_state=42)
df = df.drop(new_df.index)
df.shape, new_df.shape

((33909, 3), (200, 3))

In [15]:
#df2 = pd.read_csv('/content/drive/MyDrive/BDA_Project/Dataset1/Symptom-severity.csv')
Data = df['input_text'].tolist()
Names = df['Name'].tolist()
Queries = new_df['Uses'].tolist()

In [16]:
Data[50]

'Drug Name is Dard MR Tablet Which is used for Treatment of Pain due to muscle spasm'

In [17]:
Queries[50]

'Treatment of Depression Treatment of Anxiety disorder Treatment of Panic disorder'

In [18]:
Names[50]

'Dard MR Tablet'

### Dataset 1 loading

In [31]:
disease_symptoms_df = pd.read_csv('/content/Dataset1/symtoms_df.csv')
disease_medications_df = pd.read_csv('/content/Dataset1/medications.csv')
disease_medications_df['Medication'] = disease_medications_df['Medication'].apply(lambda x: ', '.join(eval(x)) if isinstance(x, str) else x)
df = pd.merge(disease_symptoms_df, disease_medications_df, on='Disease', how='inner')
df['BERT_Input'] = df.apply(
    lambda row: f"{row['Disease']} have symptoms {row['Symptom_1']}, {row['Symptom_2']} and {row['Symptom_3']} for which {row['Medication'].split(',')[0] if isinstance(row['Medication'], str) else row['Medication']} is used to cure.",
    axis=1
)


In [32]:
df = df.sample(frac=1, random_state=42)
new_df = df.sample(n=200, random_state=42)
df = df.drop(new_df.index)
df.shape, new_df.shape

((4240, 8), (200, 8))

In [33]:
new_df['Symptom_Text'] = new_df[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']].apply(
    lambda row: ', '.join([symptom for symptom in row if pd.notnull(symptom)][:3]),
    axis=1
)

new_df = new_df[['Disease', 'Symptom_Text']]

In [ ]:
df.head()

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Medication,BERT_Input
2451,2718,Cervical spondylosis,back_pain,weakness_in_limbs,neck_pain,dizziness,"Pain relievers, Muscle relaxants, Physical the...","Cervical spondylosis have symptoms back_pain,..."
4084,4527,Dengue,skin_rash,chills,joint_pain,vomiting,"Antibiotics, Antipyretics, Analgesics, IV flui...","Dengue have symptoms skin_rash, chills and ..."
2589,2869,Impetigo,skin_rash,high_fever,blister,red_sore_around_nose,"Topical antibiotics, Oral antibiotics, Antisep...","Impetigo have symptoms skin_rash, high_fever..."
478,548,Paralysis (brain hemorrhage),vomiting,weakness_of_one_body_side,altered_sensorium,NaN,"Blood thinners, Clot-dissolving medications, A...",Paralysis (brain hemorrhage) have symptoms vo...
1752,1942,Varicose veins,cramps,bruising,obesity,swollen_legs,"Levothyroxine, Antithyroid medications, Beta-b...","Varicose veins have symptoms cramps, bruisin..."


In [34]:
df['Name'] = df['Medication'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

In [35]:
df['Name'].head()

,Name
2451,Pain relievers
4084,Antibiotics
2589,Topical antibiotics
478,Blood thinners
1752,Levothyroxine


In [36]:
Data = df['BERT_Input'].tolist()
Names = df['Name'].tolist()
Queries = new_df['Symptom_Text'].tolist()

In [37]:
Queries[0]

' high_fever,  blister,  red_sore_around_nose'

## Embeddings

In [38]:
embeddings = generate_embeddings(Data, tokenizer, model, Bert=None)
print(embeddings.shape)
query_embeddings = generate_embeddings(Queries, tokenizer, model, query=True, Bert=None)
print(query_embeddings.shape)

(4240, 768)
(200, 768)


In [39]:
os.makedirs('/content/Dataset1/Embeddings', exist_ok=True)
np.save("/content/Dataset1/Embeddings/embeddings.npy", embeddings)
np.save("/content/Dataset1/Embeddings/query_embeddings.npy", query_embeddings)



# Recommendation

## Dataset 2 Emb REC

In [21]:
embeddings = np.load("/content/Dataset2/Embeddings/embeddings.npy")
query_embeddings = np.load("/content/Dataset2/Embeddings/query_embeddings.npy")

print(f"Loaded embeddings shape: {embeddings.shape}")
print(f"Loaded query embeddings shape: {query_embeddings.shape}")

Loaded embeddings shape: (33909, 768)
Loaded query embeddings shape: (200, 768)


### LSH

In [22]:
class LSHIndex:
    def __init__(self, hash_size=8, input_dim=None):
        """
        Initialize the LSHIndex.

        Parameters
        ----------
        hash_size : int, optional
            The hash size for LSHash, by default 8.
        input_dim : int, optional
            The dimension of the input embeddings.
            If None, it will be inferred from the given embeddings.
        """
        self.hash_size = hash_size
        self.input_dim = input_dim
        self.lsh = None
        self.texts = None

    def build_index(self, embeddings, texts):
        """
        Build the LSH index for the provided embeddings.

        Parameters
        ----------
        embeddings : np.ndarray
            A 2D array of shape (N, D), where N is the number of items and D is the embedding dimension.
        texts : list of str
            A list of texts corresponding to each embedding.

        Returns
        -------
        LSHash
            The LSHash object containing the hash tables.
        """
        embeddings = np.array(embeddings)
        if self.input_dim is None:
            self.input_dim = embeddings.shape[1]
        self.lsh = LSHash(hash_size=self.hash_size, input_dim=self.input_dim)

        for emb, txt in zip(embeddings, texts):
            self.lsh.index(emb, extra_data=txt)

        self.texts = texts
        return self.lsh

    def query(self, query_embedding, num_results=5):
        """
        Query the LSH index for similar items to a given query embedding.

        Parameters
        ----------
        query_embedding : np.ndarray
            A 1D array of shape (D,) representing the query embedding.
        num_results : int, optional
            How many nearest neighbors to return, by default 5.

        Returns
        -------
        list of dict
            A list of dictionaries containing 'text', 'distance', and 'similarity' for each retrieved item.
        """
        if self.lsh is None:
            raise ValueError("LSH index has not been built. Please call build_index() first.")

        results = self.lsh.query(query_embedding, num_results=num_results)
        results_info = []
        for result in results:
            neighbor_data = result[0]
            distance = result[1]
            neighbor_text = neighbor_data[1]
            similarity = 1.0 / (1.0 + distance)

            results_info.append({
                'text': neighbor_text,
                'distance': distance,
                'similarity': similarity
            })

        return results_info




In [25]:
# Clinical Bert Embeddings
index = 40
if __name__ == "__main__":
    lsh_index = LSHIndex(hash_size=8)
    lsh_index.build_index(embeddings, Names)
    results = lsh_index.query(query_embeddings[index], num_results=3)

print(f"Disease : {Queries[index]}")
Actual_Drug = new_df[new_df['Uses'].str.contains(f'{Queries[index]}', case=False, na=False)]['Name'].iloc[0]
print(f"Actual Drug: {Actual_Drug}")
for rec in results:
    print("Medication:", rec['text'],
          "Distance:", rec['distance'],
          "Similarity:", rec['similarity'])



Disease : Treatment of Primary biliary cirrhosis Treatment of Gallbladder stones
Actual Drug: Fortibile 300 Tablet
Medication: Ubchol Tablet Distance: 0.42896031976732374 Similarity: 0.6998094951739661
Medication: Xysolic 300 Tablet Distance: 0.46542395013586135 Similarity: 0.6823963808611758
Medication: Udiclear Tablet 300 Distance: 0.47042340466556437 Similarity: 0.6800762262264465


In [ ]:
# PubMed Embeddings

start_time = time()
index = 50
if __name__ == "__main__":
    lsh_index = LSHIndex(hash_size=8)
    lsh_index.build_index(embeddings, Names)
    results = lsh_index.query(query_embeddings[index], num_results=3)

print(f"Disease : {Queries[index]}")
Actual_Drug = new_df[new_df['Uses'].str.contains(f'{Queries[index]}', case=False, na=False)]['Name'].iloc[0]
print(f"Actual Drug: {Actual_Drug}")
for rec in results:
    print("Medication:", rec['text'],
          "Distance:", rec['distance'],
          "Similarity:", rec['similarity'])

end_time = time()
print(f"Time taken for execution: {end_time - start_time:.2f} seconds")

Disease : Treatment of Depression Treatment of Anxiety disorder Treatment of Panic disorder
Actual Drug: Essita 10 Tablet
Medication: Uprise-D3 6L Injection 1ml Distance: 356.7732183463764 Similarity: 0.0027950666755381754
Medication: Uprise-D3 6L Injection 1ml Distance: 356.77326448501617 Similarity: 0.0027950663150848177
Medication: Uprise-D3 6L Injection 1ml Distance: 356.7732663959275 Similarity: 0.002795066300156022
Time taken for execution: 4.00 seconds


### SimHash

In [26]:


def find_similar_items(text_data, query_text, n_items=3, threshold=0.8):

    sim_hash = SimHash()
    query_hash = sim_hash.get_hash(query_text)

    results = []
    for item in text_data:
        item_hash = sim_hash.get_hash(item)
        similarity = 1 - hamming(query_hash, item_hash) / sim_hash.len_hash
        if similarity >= threshold:
            results.append((item, similarity))
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:n_items]


In [27]:
start_time = time()
similar_items = find_similar_items(Data, Queries[50], n_items=3, threshold=0.9)
print(f"Items similar to {Queries[50]}: ")
for item, score in similar_items:
    print(f"\tSimilar Items: {item}\n\tSimilarity: {score:.2f}\n")
end_time = time()
print(f"Time taken for execution: {end_time - start_time:.2f} seconds")

Items similar to Treatment of Depression Treatment of Anxiety disorder Treatment of Panic disorder: 
	Similar Items: Drug Name is Citalin 10 Tablet Which is used for Treatment of Depression Treatment of Anxiety disorder Treatment of Panic disorder
	Similarity: 0.95

	Similar Items: Drug Name is Oxetin 20mg Capsule Which is used for Treatment of Depression Treatment of Panic disorder Treatment of Obsessive -compulsive disorder
	Similarity: 0.94

	Similar Items: Drug Name is Hepinz-S Tablet Which is used for Treatment of Depression Treatment of Anxiety disorder Treatment of Panic disorder
	Similarity: 0.94

Time taken for execution: 21.79 seconds


### FAISS

In [28]:
## Clinical Bert Embeddings
start_time = time()
embedding_dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dimension)
faiss_index.add(embeddings)

k = 2
D, I = faiss_index.search(query_embeddings[50:52], k)

for query_idx, (distances, indices) in enumerate(zip(D, I)):
    print(f"Results for Query {Queries[query_idx]}:")
    for rank, (distance, idx) in enumerate(zip(distances, indices)):
        print(f"  Rank {rank + 1}:")
        print(f"    Name: {Names[idx]}")
        print(f"    Distance: {distance}")

end_time = time()
print(f"Time taken for execution: {end_time - start_time:.2f} seconds")

Results for Query Treatment of Allergic conditions:
  Rank 1:
    Name: Recita 10 Tablet
    Distance: 0.38638412952423096
  Rank 2:
    Name: Recita 10 Tablet
    Distance: 0.38638412952423096
Results for Query Treatment of Dry eyes:
  Rank 1:
    Name: Ropark 2 Tablet
    Distance: 0.6600545048713684
  Rank 2:
    Name: Ropark 1 Tablet
    Distance: 0.6639074087142944
Time taken for execution: 0.13 seconds


In [ ]:
## PubMed Embeddings
start_time = time()

embedding_dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dimension)
faiss_index.add(embeddings)

k = 2
D, I = faiss_index.search(query_embeddings[50:52], k)

for query_idx, (distances, indices) in enumerate(zip(D, I)):
    print(f"Results for Query {Queries[query_idx]}:")
    for rank, (distance, idx) in enumerate(zip(distances, indices)):
        print(f"  Rank {rank + 1}:")
        print(f"    Name: {Names[idx]}")
        print(f"    Distance: {distance}")

end_time = time()
print(f"Time taken for execution: {end_time - start_time:.2f} seconds")

Results for Query Treatment of Allergic conditions:
  Rank 1:
    Name: Citalin 10 Tablet
    Distance: 100.84402465820312
  Rank 2:
    Name: Ecitelo 10 Tablet
    Distance: 104.79631042480469
Results for Query Treatment of Dry eyes:
  Rank 1:
    Name: Ropark 1 Tablet
    Distance: 81.0308609008789
  Rank 2:
    Name: Ropark 2 Tablet
    Distance: 81.34697723388672
Time taken for execution: 0.16 seconds


## Dataset 1 Emb REC

In [40]:
import numpy as np
embeddings = np.load("/content/Dataset1/Embeddings/embeddings.npy")
query_embeddings = np.load("/content/Dataset1/Embeddings/query_embeddings.npy")

print(f"Loaded embeddings shape: {embeddings.shape}")
print(f"Loaded query embeddings shape: {query_embeddings.shape}")

Loaded embeddings shape: (4240, 768)
Loaded query embeddings shape: (200, 768)


### LSH

In [41]:
# Example usage:
index = 50
if __name__ == "__main__":
    lsh_index = LSHIndex(hash_size=8)
    lsh_index.build_index(embeddings, Names)
    results = lsh_index.query(query_embeddings[index], num_results=3)

    # Print the results
    # Print your heading first:
print(f"Disease : {Queries[index]}")
for rec in results:
    print("Medication:", rec['text'],
          "Distance:", rec['distance'],
          "Similarity:", rec['similarity'])



Disease :  vomiting,  sunken_eyes,  dehydration
Medication: Analgesics Distance: 2.384138357928096 Similarity: 0.2954961926001866
Medication: Analgesics Distance: 2.384138819731854 Similarity: 0.2954961522764117
Medication: Analgesics Distance: 2.3841392046436662 Similarity: 0.2954961186666951


### FAISS

In [42]:

embedding_dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dimension)

faiss_index.add(embeddings)
k = 2
D, I = faiss_index.search(query_embeddings[40:42], k)
for query_idx, (distances, indices) in enumerate(zip(D, I)):
    print(f"Results for Query {Queries[query_idx]}:")
    for rank, (distance, idx) in enumerate(zip(distances, indices)):
        print(f"  Rank {rank + 1}:")
        print(f"    Data Index: {idx}")
        print(f"    Name: {Names[idx]}")
        print(f"    Distance: {distance}")


Results for Query  high_fever,  blister,  red_sore_around_nose:
  Rank 1:
    Data Index: 491
    Name: NSAIDs
    Distance: 1.6293201446533203
  Rank 2:
    Data Index: 3670
    Name: NSAIDs
    Distance: 1.6293201446533203
Results for Query  continuous_sneezing,  chills,  fatigue:
  Rank 1:
    Data Index: 759
    Name: Laxatives
    Distance: 1.7516061067581177
  Rank 2:
    Data Index: 964
    Name: Laxatives
    Distance: 1.7516061067581177
